In [2]:
from IPython.display import clear_output
%pip install pandas numpy torch opencv-python torchvision tqdm scikit_learn
clear_output()

In [1]:
import pandas as pd 
import numpy as np 


import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
import cv2

from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
batch_size = 64

# CNN hyperparameters
input_channels = 3  # RGB images
num_classes = 1 # Binary classification (normal vs anomaly)
learning_rate = 0.001
num_epochs = 50
dropout_rate = 0.5

# Define image dimensions
img_height = 224
img_width = 224

# Create a transform to resize images
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
])

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
# Set random seed for reproducibility
torch.manual_seed(42)

train_dataset = torchvision.datasets.ImageFolder(root='data/train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='data/test', transform=transform)

In [4]:
# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [9]:
class model(nn.Module):
    def __init__(self, input_channels, num_classes) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, 16, 3)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.
        
        self.fc1 = None
        self.fc2 = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        if self.fc1 is None:
            self.feature_size = x.shape[1] * x.shape[2] * x.shape[3]
            self.fc1 = nn.Linear(self.feature_size, 1024).to(x.device)
        
        x = torch.flatten(x, 1)  # Flatten the feature maps
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.flatten(x)

In [10]:
model = model(input_channels, num_classes).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_fn = nn.BCEWithLogitsLoss()

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch, (inputs, labels) in tqdm(enumerate(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward Pass
        outputs = model(inputs)
        loss = loss_fn(outputs, labels.float())
        
        # Backward Pass and Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        predicted = (outputs > 0).float()
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)
    
    train_loss /= len(train_loader)
    train_accuracy = 100 * train_correct / train_total

Epoch 1/50: 20it [00:11,  1.68it/s]
Epoch 2/50: 20it [00:08,  2.37it/s]
Epoch 3/50: 20it [00:08,  2.37it/s]
Epoch 4/50: 20it [00:08,  2.35it/s]
Epoch 5/50: 20it [00:08,  2.38it/s]
Epoch 6/50: 20it [00:08,  2.38it/s]
Epoch 7/50: 20it [00:08,  2.37it/s]
Epoch 8/50: 20it [00:08,  2.37it/s]
Epoch 9/50: 20it [00:08,  2.38it/s]
Epoch 10/50: 20it [00:08,  2.38it/s]
Epoch 11/50: 20it [00:08,  2.38it/s]
Epoch 12/50: 20it [00:08,  2.38it/s]
Epoch 13/50: 20it [00:08,  2.38it/s]
Epoch 14/50: 20it [00:08,  2.38it/s]
Epoch 15/50: 20it [00:08,  2.37it/s]
Epoch 16/50: 20it [00:08,  2.37it/s]
Epoch 17/50: 20it [00:08,  2.37it/s]
Epoch 18/50: 20it [00:08,  2.36it/s]
Epoch 19/50: 20it [00:08,  2.37it/s]
Epoch 20/50: 20it [00:08,  2.25it/s]
Epoch 21/50: 20it [00:08,  2.37it/s]
Epoch 22/50: 20it [00:08,  2.38it/s]
Epoch 23/50: 20it [00:08,  2.32it/s]
Epoch 24/50: 20it [00:08,  2.38it/s]
Epoch 25/50: 20it [00:08,  2.38it/s]
Epoch 26/50: 20it [00:08,  2.37it/s]
Epoch 27/50: 20it [00:08,  2.37it/s]
Epoch 28/5

In [ ]:
test_loss = 0
test_correct = 0
test_total = 0

with torch.no_grad():
    model.eval()
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, labels.float())
        test_loss += loss.item()
        predicted = (outputs > 0.5).float()  # binary classification
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

test_loss /= len(test_loader)
test_accuracy = 100 * test_correct / test_total

print(f"Test Accuracy: {test_accuracy:.2f}%, Test Loss: {test_loss:.4f}")

Test Accuracy: 88.82%, Test Loss: 0.3776
